In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F

In [2]:
wine_data = pd.read_csv('data/wine_data.csv')
wine_data.sample(5)

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
81,1,12.72,1.81,2.20,18.8,86,2.20,2.53,0.26,1.77,3.90,1.16,3.14,714
161,2,13.69,3.26,2.54,20.0,107,1.83,0.56,0.50,0.80,5.88,0.96,1.82,680
63,1,12.37,1.13,2.16,19.0,87,3.50,3.10,0.19,1.87,4.45,1.22,2.87,420
138,2,13.49,3.59,2.19,19.5,88,1.62,0.48,0.58,0.88,5.70,0.81,1.82,580
53,0,13.77,1.90,2.68,17.1,115,3.00,2.79,0.39,1.68,6.30,1.13,2.93,1375


#### Collecting Features

In [3]:
wine_features = wine_data.drop('Class', axis = 1)
wine_target = wine_data[['Class']]

In [4]:
from sklearn.model_selection import train_test_split

X_train, x_test, Y_train, y_test = train_test_split(wine_features,
                                                    wine_target,
                                                    test_size=0.4,
                                                    random_state=0)

In [5]:
Xtrain_ = torch.from_numpy(X_train.values).float()
Xtest_ = torch.from_numpy(x_test.values).float()

In [6]:
Ytrain_ = torch.from_numpy(Y_train.values).view(1,-1)[0]
Ytest_ = torch.from_numpy(y_test.values).view(1,-1)[0]

## Creating a classifier


In [7]:
input_size = 13
output_size = 3
hidden_size = 100

In [8]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, X):
        X = torch.sigmoid((self.fc1(X)))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)

        return F.log_softmax(X, dim=-1)

In [9]:
model = Net()

In [10]:
optimizer = optim.Adam(model.parameters(), lr = 0.01)
loss_fn = nn.NLLLoss()

#### Training the model

In [11]:
epochs = 100

for epoch in range(epochs):

    optimizer.zero_grad()
    Ypred = model(Xtrain_)

    loss = loss_fn(Ypred , Ytrain_)
    loss.backward()

    optimizer.step()
        
    if epoch % 10 == 0:
        print ('Epoch', epoch, 'loss', loss.item())
        

Epoch 0 loss 1.1079745292663574
Epoch 10 loss 1.0101490020751953
Epoch 20 loss 0.7331444621086121
Epoch 30 loss 0.6127467751502991
Epoch 40 loss 0.5867196917533875
Epoch 50 loss 0.5545439720153809
Epoch 60 loss 0.5383915901184082
Epoch 70 loss 0.4714454114437103
Epoch 80 loss 0.4493368864059448
Epoch 90 loss 0.35973823070526123


In [12]:
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
    }, 'wine_checkpoint.pth')

In [13]:
predict_out = model(Xtest_)
_, predict_y = torch.max(predict_out, 1)

In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

print ('prediction accuracy', accuracy_score(Ytest_.data, predict_y.data))
print ('micro precision', precision_score(Ytest_.data, predict_y.data, average='micro'))
print ('micro recall', recall_score(Ytest_.data, predict_y.data, average='micro'))

prediction accuracy 0.8194444444444444
micro precision 0.8194444444444444
micro recall 0.8194444444444444
